In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

from sklearn.model_selection import train_test_split 

# Run Queries

In [2]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'BOG'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=20)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 1

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [3]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",138674,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [4]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",48322,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
frida_products = frida_products[['upc','id_tarjeta','categoria','subcategoria']].copy()

df_margen = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144493,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_margen = pd.merge(df_margen, frida_products,  how='left', left_on=['sku'], right_on = ['upc'])
df_margen[["gmv_value","cogs_value"]] = df_margen[["gmv_value","cogs_value"]].astype(float)

200
200


In [5]:
df_margen['day'] = pd.to_datetime(df_margen['day'])
max_index = df_margen.groupby('id_tarjeta')['day'].idxmax()
df_margen = df_margen.loc[max_index].copy()

In [6]:
# calculate the profit margin by sku
df_margen2 = df_margen.groupby(by=['region','id_tarjeta']).apply(lambda x: (x['gmv_value'].sum() - x['cogs_value'].sum()) / x['gmv_value'].sum() if x['gmv_value'].sum() != 0 else 0).reset_index(name='GM')

# calculate the profit margin by subcategory
df_margen3 = df_margen.groupby(by=['region','subcategoria']).apply(lambda x: (x['gmv_value'].sum() - x['cogs_value'].sum()) / x['gmv_value'].sum()).reset_index(name='GM')

/home/jupyter-felipe.guevara/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [7]:
query = """
WITH exploded_data AS (
SELECT DISTINCT
    bs.site_identifier_value AS region_code,
    bo.offer_id,
    MAX(bcs.customer_segment_id) AS customer_segment_id_test,
    --bcs.customer_segment_name,
    MAX(bcs2.customer_segment_id) AS customer_segment_id_control,
    --bcs2.customer_segment_name,
    bo.offer_name AS discount,
    DATE(bo.start_date) AS start_date,
    DATE(bo.end_date) AS end_date,
    bo.offer_value::float as Value_Discount,
    replace(
        substring(
          boic.order_item_match_rule, 
          position('["' in boic.order_item_match_rule) + 2, 
          position('"]' in boic.order_item_match_rule) - position('["' in boic.order_item_match_rule) - 2
    ), '"', '') AS values_string
    
FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"      btcofx  ON (btcofx.offer_id = bo.offer_id)
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id and (boic.archived is null or boic.archived ='N'))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs     ON bocsx.customer_segment_id = bcs.customer_segment_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs2    ON bcs.customer_segment_name||'_CONTROL' = bcs2.customer_segment_name

WHERE offer_name ILIKE 'ELASTICITY%'
    AND end_date >= DATE_TRUNC('year', date(getdate()) - interval '0 year')
    AND end_date >= DATE_TRUNC('month', date(getdate()) - interval '1 month')
    --AND start_date >= DATE_TRUNC('month', date(getdate()) - interval '0 month')
    AND bo.offer_discount_type = 'PERCENT_OFF'
    AND bo.offer_type = 'ORDER_ITEM'
    AND bo.sndbx_id is NULL
    AND (bo.archived is NULL OR bo.archived='N')
    AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
    AND bo.automatically_added = 'true'
    --AND bo.offer_id <> -14489998
    --AND bs.site_identifier_value = 'BOG'
GROUP BY 1,2,5,6,7,8,9
), 
 
number_series AS (
  SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10
  UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14 UNION ALL SELECT 15 UNION ALL SELECT 16 UNION ALL SELECT 17 UNION ALL SELECT 18 UNION ALL SELECT 19 UNION ALL SELECT 20
  UNION ALL SELECT 21 UNION ALL SELECT 22 UNION ALL SELECT 23 UNION ALL SELECT 24 UNION ALL SELECT 25 UNION ALL SELECT 26 UNION ALL SELECT 27 UNION ALL SELECT 28 UNION ALL SELECT 29 UNION ALL SELECT 30
  UNION ALL SELECT 31 UNION ALL SELECT 32 UNION ALL SELECT 33 UNION ALL SELECT 34 UNION ALL SELECT 35 UNION ALL SELECT 36 UNION ALL SELECT 37 UNION ALL SELECT 38 UNION ALL SELECT 39 UNION ALL SELECT 40
  UNION ALL SELECT 41 UNION ALL SELECT 42 UNION ALL SELECT 43 UNION ALL SELECT 44 UNION ALL SELECT 45 UNION ALL SELECT 46 UNION ALL SELECT 47 UNION ALL SELECT 48 UNION ALL SELECT 49 UNION ALL SELECT 50
  UNION ALL SELECT 51 UNION ALL SELECT 52 UNION ALL SELECT 53 UNION ALL SELECT 54 UNION ALL SELECT 55 UNION ALL SELECT 56 UNION ALL SELECT 57 UNION ALL SELECT 58 UNION ALL SELECT 59 UNION ALL SELECT 60
  UNION ALL SELECT 61 UNION ALL SELECT 62 UNION ALL SELECT 63 UNION ALL SELECT 64 UNION ALL SELECT 65 UNION ALL SELECT 66 UNION ALL SELECT 67 UNION ALL SELECT 68 UNION ALL SELECT 69 UNION ALL SELECT 70
  UNION ALL SELECT 71 UNION ALL SELECT 72 UNION ALL SELECT 73 UNION ALL SELECT 74 UNION ALL SELECT 75 UNION ALL SELECT 76 UNION ALL SELECT 77 UNION ALL SELECT 78 UNION ALL SELECT 79 UNION ALL SELECT 80
  UNION ALL SELECT 81 UNION ALL SELECT 82 UNION ALL SELECT 83 UNION ALL SELECT 84 UNION ALL SELECT 85 UNION ALL SELECT 86 UNION ALL SELECT 87 UNION ALL SELECT 88 UNION ALL SELECT 89 UNION ALL SELECT 90
  UNION ALL SELECT 91 UNION ALL SELECT 92 UNION ALL SELECT 93 UNION ALL SELECT 94 UNION ALL SELECT 95 UNION ALL SELECT 96 UNION ALL SELECT 97 UNION ALL SELECT 98 UNION ALL SELECT 99 UNION ALL SELECT 100
)

SELECT DISTINCT
    ed.region_code,
    ed.offer_id,
    ed.customer_segment_id_test,
    ed.customer_segment_id_control,
    ed.discount,
    ed.start_date,
    ed.end_date,
    ed.value_discount,
    bs.addl_product_id AS card_id,
    bs2.name,
    bcat2.name AS cat,
    bcat.name as subcat
    
FROM exploded_data                                                                  ed
JOIN number_series                                                                  ns      ON ns.n <= regexp_count(ed.values_string, ',') + 1 AND trim(split_part(ed.values_string, ',', ns.n)) <> ''
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = trim(split_part(ed.values_string, ',', ns.n))::int
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat2   ON bcx.category_id = bcat2.category_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref"   bcs     ON bo.customer_segment_id = bcs.customer_segment_id

WHERE
    -- FIX SUPER DESCUENTOS
    bcat2.category_id NOT IN ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
"""#.format(clientes=tuple(set(clear_buyers[clear_buyers.registered_city == city].customer_id.unique())), city = city)

df_offers = read_connection_data_warehouse.runQuery(query)
df_offers['start_date'] = pd.to_datetime(df_offers['start_date'])
df_offers['end_date'] = pd.to_datetime(df_offers['end_date'])

In [8]:
df_offers = df_offers.loc[df_offers.offer_id.isin([-14513772,-14513773,-14513774])].reset_index(drop=True)

In [9]:
df_offers.loc[df_offers.region_code == city_code]

,region_code,offer_id,customer_segment_id_test,customer_segment_id_control,discount,start_date,end_date,value_discount,card_id,name,cat,subcat
0,BOG,-14513772,170754,170753,ELASTICITY_2023-05-31_BOG_Cebolla Cabezona Bla...,2023-06-01,2023-06-08,6.0,-200012,Cebolla Cabezona Blanca Sin Pelar,Frutas & Verduras,Verduras
1,BOG,-14513774,170803,170802,ELASTICITY_2023-05-31_BOG_Limón Tahití_ELASTIC,2023-06-01,2023-06-08,6.0,-200028,Limón Tahití,Frutas & Verduras,Frutas
2,BOG,-14513773,170751,170752,ELASTICITY_2023-05-31_BOG_Cebolla Cabezona Bla...,2023-06-01,2023-06-08,6.0,-200012,Cebolla Cabezona Blanca Sin Pelar,Frutas & Verduras,Verduras


In [10]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    --ffg.close_date
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment" ba          ON ba.order_id = bo.order_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND bo.submit_date >= DATE_TRUNC('week', date(getdate()) - interval '3 week')
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{ciudad}'
    
--GROUP BY bo.order_id, ffg.close_date
)

SELECT DISTINCT
    s.site_identifier_value AS city,
    TO_CHAR(DATE_TRUNC('week', DATE_TRUNC('day', bo.submit_date) + INTERVAL '4 day'), 'WW') AS week,
    --TO_CHAR(bo.submit_date, 'WW') AS week,
    bo.customer_id,
    bs2.name,
    bs.addl_product_id as padre_sku_id,
    bcat2.name AS category,
    bcat.name AS subcat,
    MIN(bo.submit_date) AS min_date,
    COUNT(DISTINCT bo.order_id) as orders,
    CASE WHEN s.site_identifier_value = 'CMX' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'GDL' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'PBC' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'MTY' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
        WHEN s.site_identifier_value = 'SPO' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'BHZ' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'CWB' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
        WHEN s.site_identifier_value = 'VCP' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    ELSE SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/3776) END AS gmv_usd,
    CASE WHEN s.site_identifier_value = 'CMX' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'GDL' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'PBC' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'MTY' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
        WHEN s.site_identifier_value = 'SPO' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
        WHEN s.site_identifier_value = 'BHZ' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
        WHEN s.site_identifier_value = 'CWB' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
        WHEN s.site_identifier_value = 'VCP' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    ELSE (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/3776) END AS discount_usd,
    SUM(boi.quantity* foi.step_unit) as cant

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"     boipd   ON boipd.order_item_id=boi.order_item_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"       boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
    AND (LOWER(boida.adjustment_reason) ILIKE '%hook%') OR (LOWER(boida.adjustment_reason) ILIKE '%elasticity%')
    
GROUP BY 1,2,3,4,5,6,7
""".format(ciudad=city_code)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd","discount_usd","cant"]] = data_ventas_aux[["gmv_usd","discount_usd","cant"]].astype(float)
#data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])
#data_ventas_aux = data_ventas_aux.loc[data_ventas_aux.week.isin(['05','06','07'])].reset_index(drop=True)

In [11]:
data_ventas_aux.groupby('week').min_date.min()

week
20   2023-05-15 00:00:10.989
21   2023-05-18 00:00:30.687
22   2023-05-25 00:00:05.804
23   2023-06-01 00:00:06.285
24   2023-06-08 00:01:09.066
Name: min_date, dtype: datetime64[ns]

# Cleaning Data and some functions

In [12]:
if city_code == 'BOG':
    skus_exp = [-200028,-200012]

In [13]:
df_exp = data_ventas_aux.loc[data_ventas_aux.padre_sku_id.isin(skus_exp)]

df_exp =  pd.merge(df_exp, df_margen2.loc[df_margen2.region == city_code].reset_index(drop=True),  how='left', left_on=['padre_sku_id'], right_on = ['id_tarjeta'])

df_exp['cash_margin'] = df_exp.gmv_usd * df_exp.GM

df_exp = df_exp.loc[:,['city','week','customer_id','name','padre_sku_id','category','subcat','orders','gmv_usd','discount_usd','cant','GM','cash_margin']]

In [14]:
all_weeks = data_ventas_aux.groupby('week').customer_id.nunique()

In [15]:
all_weeks

week
20    5132
21    7348
22    7983
23    9409
24    2489
Name: customer_id, dtype: int64

In [16]:
def customers_offer(grupo):
    query = """
    SELECT DISTINCT 
        bcs.customer_segment_id, 
        bcs.external_identifier::int AS customer_id
    FROM postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref" bcs
    WHERE bcs.customer_segment_id = {clientes}
    """.format(clientes=grupo)

    df_clients = read_connection_data_warehouse.runQuery(query)
    return df_clients.customer_id.unique()

In [17]:
def df_info_filter_offer(df_offers_grouped, i, week, test, control):
    df_filter_test = df_exp[(df_exp['city'] == df_offers_grouped.loc[i,'region_code']) & 
                         (df_exp['customer_id'].isin(test)) & 
                         (df_exp['week'] == week) & 
                         (df_exp['padre_sku_id'] == df_offers_grouped.loc[i,'card_id'])].copy()
    df_filter_control = df_exp[(df_exp['city'] == df_offers_grouped.loc[i,'region_code']) & 
                         (df_exp['customer_id'].isin(control)) & 
                         (df_exp['week'] == week) & 
                         (df_exp['padre_sku_id'] == df_offers_grouped.loc[i,'card_id'])].copy()
    
    if len(df_filter_control) == 0:
        dic = {}
        dic = df_filter_test.iloc[0].to_dict()
        dic['week'] = week
        dic['customer_id'] = 0
        dic['gmv_usd'] = 0
        dic['discount_usd'] = 0
        dic['cant'] = 0
        new_df = pd.DataFrame(dic, index=[0])
        df_filter_control = pd.concat([df_filter_control, new_df], ignore_index=True)

    if len(df_filter_test) == 0:
        dic = {}
        dic = df_filter_control.iloc[0].to_dict()
        dic['week'] = week
        dic['customer_id'] = 0
        dic['gmv_usd'] = 0
        dic['discount_usd'] = 0
        dic['cant'] = 0
        new_df = pd.DataFrame(dic, index=[0])
        df_filter_test = pd.concat([df_filter_test, new_df], ignore_index=True)
        
    df_filter_test['group'] = 'test'
    df_filter_control['group'] = 'control'
    df_filter_test['all_customer'] = len(test)
    df_filter_control['all_customer'] = len(control)
    df_filter = pd.concat([df_filter_test, df_filter_control]).reset_index(drop=True)
    df_filter['offer_id'] = df_offers_grouped.loc[i,'offer_id']
    df_filter['GM'] = df_filter.padre_sku_id.apply(lambda x: 0.1 if len(df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()) == 0 else df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()[0])
    #df_filter['GM'] = df_filter.padre_sku_id.apply(lambda x: df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()[0])
    df_filter['cash_margin'] = df_filter.gmv_usd * df_filter.GM
    df_filter['net_cash_margin'] = df_filter.cash_margin - df_filter.discount_usd
    return df_filter

In [18]:
df_exp=df_exp.loc[df_exp.week.isin(['21','22','23'])]

In [20]:
df_exp

,city,week,customer_id,name,padre_sku_id,category,subcat,orders,gmv_usd,discount_usd,cant,GM,cash_margin
0,BOG,23,265676944,Cebolla Cabezona Blanca Sin Pelar,-200012,Frutas & Verduras,Verduras,2,7.035169,0.211050,17.5,0.289835,2.039038
2,BOG,23,-249330,Limón Tahití,-200028,Frutas & Verduras,Frutas,1,4.136282,0.248159,15.0,0.388534,1.607086
4,BOG,23,37365422,Limón Tahití,-200028,Frutas & Verduras,Frutas,2,7.908527,0.474497,27.5,0.388534,3.072733
5,BOG,23,-221220,Cebolla Cabezona Blanca Sin Pelar,-200012,Frutas & Verduras,Verduras,1,5.028469,0.301708,12.5,0.289835,1.457426
6,BOG,23,12212970,Cebolla Cabezona Blanca Sin Pelar,-200012,Frutas & Verduras,Verduras,1,1.005694,0.030171,2.5,0.289835,0.291485
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,BOG,23,253905209,Cebolla Cabezona Blanca Sin Pelar,-200012,Frutas & Verduras,Verduras,1,1.003350,0.030098,2.5,0.289835,0.290806
5566,BOG,23,40524548,Cebolla Cabezona Blanca Sin Pelar,-200012,Frutas & Verduras,Verduras,1,1.003350,0.030098,2.5,0.289835,0.290806
5567,BOG,23,43014928,Limón Tahití,-200028,Frutas & Verduras,Frutas,2,15.730932,0.471928,40.0,0.388534,6.112003
5568,BOG,23,-224198,Cebolla Cabezona Blanca Sin Pelar,-200012,Frutas & Verduras,Verduras,1,4.337924,0.130138,15.0,0.289835,1.257282


In [19]:
results_df = pd.DataFrame()
for i in range(len(df_offers)):
    print(df_offers.loc[i,'offer_id'])
    test = customers_offer(df_offers.loc[i,'customer_segment_id_test'])
    control = customers_offer(df_offers.loc[i,'customer_segment_id_control'])
    for week in df_exp.week.unique():
        info_offer = df_info_filter_offer(df_offers, i, week, test, control)
        results_offer = (info_offer.groupby(by=['city', 'week','offer_id', 'name', 'group', 'all_customer'])
                            .agg(customer_count=('customer_id', 'nunique'),
                                gmv_usd=('gmv_usd', np.sum),
                                discount_usd=('discount_usd', np.sum),
                                quantity=('cant', np.sum),
                                margen=('GM', np.mean),
                                cash_margin=('cash_margin', np.sum),
                                net_cash_margin=('net_cash_margin', np.sum))
                            .reset_index()
                )
    
        results_offer['convertion_rate'] = results_offer.customer_count / results_offer.all_customer       
        results_offer['AOV'] = results_offer.gmv_usd / results_offer.customer_count
        results_offer['AOV_GM'] = results_offer.cash_margin / results_offer.customer_count
        results_offer['dct_avg'] = results_offer.discount_usd / results_offer.customer_count
        
        results_df = pd.concat([results_df, results_offer]).reset_index(drop=True)

-14513772
-14513774


IndexError: single positional indexer is out-of-bounds

In [ ]:
#import re

#pattern = re.compile(r'_(STA|GRU|VLP)_')
# extract the warehouse name from the column and create a new column in the dataframe
#df_offers['wh'] = df_offers['discount'].apply(lambda x: pattern.search(x).group(1))

In [ ]:
results_df = pd.merge(results_df, df_offers[['offer_id','discount']],  how='left', left_on=['offer_id'], right_on = ['offer_id'])

In [ ]:
results_df.name.unique()

array(['Azúcar Morena Estándar x 50 kg'], dtype=object)

In [ ]:
results_df.loc[(results_df.name == 'Azúcar Morena Estándar x 50 kg') & (results_df.week == '20')].to_excel("info_MX.xlsx")